In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import xgboost as xgb
from xgboost import XGBRegressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, classification_report, accuracy_score
from sklearn.preprocessing import RobustScaler

from sklearn.neighbors import BallTree

import itertools

In [3]:
#read csv
avg_trips = pd.read_csv('./processed_data/avg_trips_forprediction.csv')


avg_trips

,day_of_week,hour,Seccion_init,Seccion_end,avg_trips,stratify_group,log_avg_trips,Edad media de la población,Porcentaje de población menor de 18 años,Porcentaje de población de 65 y más años,...,area_m2,neighbor_count,area_m2_end,neighbor_count_end,distance_km,hour_c,Seccion_init_c,Seccion_end_c,num_stations_c,num_stations_end_c
0,5,-1.0,-1.000998,-1.000998,1.0,0.0,69.314718,-0.107143,0.081967,-0.033557,...,-0.272798,-0.043478,-0.272798,-0.043478,-1.177165,0,2006901002,2006901002,1.0,1.0
1,5,-1.0,-1.000998,-0.999335,0.0,-1.0,0.000000,-0.107143,0.081967,-0.033557,...,-0.272798,-0.043478,-0.363434,0.173913,-1.018029,0,2006901002,2006901007,1.0,1.0
2,5,-1.0,-1.000998,-0.668440,0.0,-1.0,0.000000,-0.107143,0.081967,-0.033557,...,-0.272798,-0.043478,0.402557,0.434783,-0.991054,0,2006901002,2006902002,1.0,2.0
3,5,-1.0,-1.000998,-0.667775,0.0,-1.0,0.000000,-0.107143,0.081967,-0.033557,...,-0.272798,-0.043478,-0.215694,0.869565,-0.845172,0,2006901002,2006902004,1.0,1.0
4,5,-1.0,-1.000998,-0.667443,0.0,-1.0,0.000000,-0.107143,0.081967,-0.033557,...,-0.272798,-0.043478,-0.361918,0.521739,-0.859509,0,2006901002,2006902005,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310627,3,1.0,1.004988,0.671101,0.0,-1.0,0.000000,0.535714,-0.262295,0.369128,...,0.000000,-0.521739,11.762436,-1.130435,2.249354,23,2006907034,2006906030,1.0,1.0
310628,3,1.0,1.004988,0.996342,1.0,0.0,69.314718,0.535714,-0.262295,0.369128,...,0.000000,-0.521739,0.206107,-0.130435,-0.490196,23,2006907034,2006907008,1.0,1.0
310629,3,1.0,1.004988,1.002660,0.0,-1.0,0.000000,0.535714,-0.262295,0.369128,...,0.000000,-0.521739,8.951174,-1.130435,0.221156,23,2006907034,2006907027,1.0,1.0
310630,3,1.0,1.004988,1.004656,1.0,0.0,69.314718,0.535714,-0.262295,0.369128,...,0.000000,-0.521739,0.393618,-0.521739,-0.941262,23,2006907034,2006907033,1.0,1.0


In [4]:
avg_trips.columns

Index(['day_of_week', 'hour', 'Seccion_init', 'Seccion_end', 'avg_trips',
       'stratify_group', 'log_avg_trips', 'Edad media de la población',
       'Porcentaje de población menor de 18 años',
       'Porcentaje de población de 65 y más años', 'Tamaño medio del hogar',
       'Porcentaje de hogares unipersonales', 'Población',
       'Porcentaje de población española', 'Renta neta media por persona',
       'Renta neta media por hogar', 'Media de la renta por unidad de consumo',
       'Mediana de la renta por unidad de consumo',
       'Renta bruta media por persona', 'Renta bruta media por hogar',
       'Edificabil', 'Solar', 'Vivienda', 'Trastero', 'Garaje Cer',
       'Agropecuar', 'Local Come', 'Industrial', 'Almacen', 'Deportivo',
       'Cultural', 'Educativo', 'Oficina', 'Hosteleria', 'Obras Urba',
       'Altura men', 'Terraza', 'Accesos y', 'Jardin', 'Sanidad', 'Deposito',
       'Duplex', 'Plaza Park', 'Religioso', 'Espectacul', 'Renta_medi',
       'num_stations', 'Eda

In [5]:
features = avg_trips.columns.difference(["avg_trips","log_avg_trips","weights","stratify_group",'hour_c', 'Seccion_init_c', 'Seccion_end_c', 'num_stations_c', 'num_stations_end_c'])
target = "log_avg_trips"

data = avg_trips
X = data[features]
y = data[target]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model Training
xgb_model = XGBRegressor(n_estimators=300, learning_rate=0.1, subsample=0.8, max_depth=7, random_state=42)
xgb_model.fit(X_train, y_train, )

# Predictions
y_pred = xgb_model.predict(X_test)

# Evaluation
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R² Score: {r2}")

Root Mean Squared Error (RMSE): 23.25686265229781
R² Score: 0.60145693467783


In [6]:
# Create buckets for actual and predicted values
y_test_buckets = pd.Series(y_test.values, index=y_test.index).apply(
    lambda x: 0 if x == 0 else (1 if x == 1 else 2)
)
y_pred_buckets = pd.Series(y_pred, index=y_test.index).apply(
    lambda x: 0 if x < 0.5 else (1 if x < 1.5 else 2)
)

# Calculate classification-like accuracy

classification_accuracy = accuracy_score(y_test_buckets, y_pred_buckets)

print(f"Classification-like Accuracy (Buckets): {classification_accuracy * 100:.2f}%")

print(classification_report(y_test_buckets, y_pred_buckets, target_names=["Zero Trips", "One Trip", "> One Trip"]))



Classification-like Accuracy (Buckets): 67.97%
              precision    recall  f1-score   support

  Zero Trips       0.99      0.20      0.34     24928
    One Trip       0.00      0.00      0.00         0
  > One Trip       0.67      1.00      0.80     37199

    accuracy                           0.68     62127
   macro avg       0.55      0.40      0.38     62127
weighted avg       0.80      0.68      0.62     62127



/Users/naroacorettisanchez/anaconda3/envs/ml/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/naroacorettisanchez/anaconda3/envs/ml/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/naroacorettisanchez/anaconda3/envs/ml/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.cap

In [8]:
#unique values for 'day of week' and 'hour'ArithmeticError
unique_days = avg_trips['day_of_week'].unique()
unique_hours = avg_trips['hour_c'].unique()
print("Unique days of the week:", unique_days)
print("Unique hours:", unique_hours)

Unique days of the week: [5 1 6 7 4 2 3]
Unique hours: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]


In [10]:
avg_trips['num_stations_end_c'].value_counts()

num_stations_end_c
1.0    252840
2.0     50568
3.0      7224
Name: count, dtype: int64